In [26]:
import pandas as pd

In [27]:
df = pd.read_csv("workouts.csv")

In [28]:
df.columns =["prompt", "completion"]

In [29]:
df.head()

,prompt,completion
0,"I've injured my ankle and foot, but I'd still ...","If your ankle or foot is injured, you still ha..."
1,I'd like two circuit training routines that I ...,Leg and knee injuries can be fairly limiting f...
2,I would like an exercise circuit that I can do...,Shoulder or other upper body injuries often al...
3,I would like general advice about exercising w...,Back injuries can be difficult to recover from...
4,I'm a runner and I would like some exercises a...,"1. Pool running\nAlso known as aqua jogging, i..."


In [30]:
def create_fine_tuning_df(df):
    '''
    Format workouts dataset according to openai best practice for fine tuning
    
    Parameters
    ----------
    df: pd.DataFrame
        The dataframe containing human made prompt, completion pairs
    
    Returns
    -------
    pd.DataFrame
        The reformatted dataframe
    '''
    rows = []
    for i, row in df.iterrows():
        rows.append({"prompt":f"{row.prompt} \n\n###\n\n", "completion":f" {row.completion} ###"})
    return pd.DataFrame(rows)

fine_tuning_df = create_fine_tuning_df(df)

In [32]:
fine_tuning_df.to_json(f'ft_data.jsonl', orient='records', lines=True)

Run the openai fine tuning tool to suggests improvements in your data formatting for fine-tuning.

In [33]:
!openai tools fine_tunes.prepare_data -f ft_data.jsonl

Logging requires wandb to be installed. Run `pip install wandb`.
Analyzing...

- Your file contains 59 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- All prompts end with suffix ` \n\n###\n\n`
- All completions end with suffix ` ###`

No remediations found.

You can use your file for fine-tuning:
> openai api fine_tunes.create -t "ft_data.jsonl"

After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string ` \n\n###\n\n` for the model to start generating completions, rather than continuing with the prompt. Make sure to include `stop=[" ###"]` so that the generated texts ends at the expected place.
Once your model starts training, it'll approximately take 3.25 minutes to train a `curie` model, and less for `ada` and `babbage`. Queue will approximately take half an hour per job ahead of you.


In [34]:
openai api fine_tunes.create -t ft_data.jsonl -m davinci --suffix "workout_generator_m1"

Logging requires wandb to be installed. Run `pip install wandb`.
Upload progress: 100%|████████████████████| 76.9k/76.9k [00:00<00:00, 26.2Mit/s]
Uploaded file from ft_data.jsonl: file-p3hbn0BlkQRFPwwbrrskLdw5
Created fine-tune: ft-sz4DVDfgMRCaHLBiu7XBr4nB
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-04-25 12:42:56] Created fine-tune: ft-sz4DVDfgMRCaHLBiu7XBr4nB
[2022-04-25 12:43:10] Fine-tune costs $2.37
[2022-04-25 12:43:11] Fine-tune enqueued. Queue number: 0
[2022-04-25 12:43:15] Fine-tune started
[2022-04-25 12:46:55] Completed epoch 1/4
[2022-04-25 12:47:26] Completed epoch 2/4
[2022-04-25 12:47:57] Completed epoch 3/4
[2022-04-25 12:48:28] Completed epoch 4/4
[2022-04-25 12:49:01] Uploaded model: davinci:ft-personal:workout-generator-v1-2022-04-25-16-48-59
[2022-04-25 12:49:04] Uploaded result file: file-mDaWvTvgkNfk8Mt8zyTLoeEC
[2022-04-25 12:49:05] Fine-tune succeeded

Job complete! Status: succeeded 

In [38]:
!openai api completions.create -m davinci:ft-personal:workout-generator-v1-2022-04-25-16-48-59 -p squats

Logging requires wandb to be installed. Run `pip install wandb`.
squats, and inverted rows. Choose those exercises, or better yet, use the

In [39]:
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

In [43]:
prompt ="i want to do exercises to increase the size of my butt \n\n###\n\n"
model = "davinci:ft-personal:workout-generator-v1-2022-04-25-16-48-59"

In [45]:
response = openai.Completion.create(model=model, prompt=prompt, stop=[" ###"], temperature=0, max_tokens=2000)

In [46]:
response

<OpenAIObject text_completion id=cmpl-50xHAauMAove41eIodx2VjQm7dXxH at 0x1469ec810> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " Sissy Squats\n\nBanded Hip Hinge\n\nBanded Hip Hinge\n\nGlute Bridge\n\nGlute Bridge\n\nDumbbell Single Leg Deadlift\n\nDumbbell Single Leg Deadlift\n\nDumbbell Stiff Leg Deadlift\n\nDumbbell Stiff Leg Deadlift\n\nDumbbell Single Leg Deadlift\n\nDumbbell Single Leg Deadlift\n\nGlute Bridge March\n\nGlute Bridge March\n\nBanded Hip Hinge\n\nBanded Hip Hinge\n\nBanded Single Leg Deadlift\n\nBanded Single Leg Deadlift\n\nBanded Single Leg Deadlift\n\nBanded Single Leg Deadlift\n\nPelvic Tilt\n\nPelvic Tilt\n\nPelvic Tilt\n\nPelvic Tilt"
    }
  ],
  "created": 1650909540,
  "id": "cmpl-50xHAauMAove41eIodx2VjQm7dXxH",
  "model": "davinci:ft-personal:workout-generator-v1-2022-04-25-16-48-59",
  "object": "text_completion"
}

In [47]:
df.head()

,prompt,completion
0,"I've injured my ankle and foot, but I'd still ...","If your ankle or foot is injured, you still ha..."
1,I'd like two circuit training routines that I ...,Leg and knee injuries can be fairly limiting f...
2,I would like an exercise circuit that I can do...,Shoulder or other upper body injuries often al...
3,I would like general advice about exercising w...,Back injuries can be difficult to recover from...
4,I'm a runner and I would like some exercises a...,"1. Pool running\nAlso known as aqua jogging, i..."


In [53]:
df['completion'][12]

"Full-Body Workout 1\nThe first two movements for this session are the barbell hip thrust and the hyperextension. These are two very effective movements if they are done properly.\n\nBarbell Hip Thrust – Make sure the barbell is in a position that you’re comfortable with. If you need a towel or mat around the bar, don’t be afraid to use it. The more comfortable you can be, the more beneficial the exercise is. When you lift the barbell up, make sure the glutes contract hard. Hold that position before slowly lowering the weight back down.\n\nHyperextension - This is considered by many to be a lower back exercise. If you stop when your upper body is at a 45-degree angle, then it will isolate the lower back. Raise your upper body up, and then contract the glutes so the upper body comes up even more. Your torso should be close to vertical before you lower yourself back down. If you don’t have a hyperextension bench, a preacher bench or adjustable bench would work as well.\n\nThe rest of the

In [ ]:
df[]